In [1]:
# pip install chromadb

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv('./.env')
openai_api_key = os.getenv("OPENAI_API_KEY")


In [2]:
llm = ChatOpenAI(
    model='gpt-4o-mini',
    openai_api_key = openai_api_key,
    max_tokens = 512
)

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA

# Configurar as embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [4]:
# Criar um banco de dados vetorial usando Chroma
vector_store = Chroma(collection_name="minha_colecao", embedding_function=embeddings)


C:\Users\otimi\AppData\Local\Temp\ipykernel_48284\175187644.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(collection_name="minha_colecao", embedding_function=embeddings)


In [5]:
# Adicionar documentos
documentos = [
    Document(page_content="Maria foi a feira.", metadata={"id": "1"}),
    Document(page_content="João foi na escola.", metadata={"id": "2"}),
    Document(page_content="José foi na universidade.", metadata={"id": "3"}),
    Document(page_content="Gustavo ficou em casa.", metadata={"id": "4"}),
    Document(page_content="Olívia foi na farmácia.", metadata={"id": "5"}),
]
vector_store.add_documents(documentos)

# Configurar o sistema de recuperação
retriever = vector_store.as_retriever()


In [6]:
# Criar uma chain de QA com o modelo da OpenAI
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever
)

# Executar uma consulta
query = "Onde está Maria? Mostre o documento que diz isso."
resposta = qa.invoke(query)
print(resposta)

{'query': 'Onde está Maria? Mostre o documento que diz isso.', 'result': 'Maria foi à feira.'}


In [7]:
from langchain_community.document_loaders import TextLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('dados/lei9279.txt', encoding='utf-8') # to load text document 
documents = loader.load() 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) 
chunks = text_splitter.split_documents(documents) 


In [8]:
chunks[20]

Document(metadata={'source': 'dados/lei9279.txt'}, page_content='Parágrafo único. O pedido de patente poderá ser desarquivado, se o depositante assim o requerer, dentro de 60 (sessenta) dias contados do arquivamento, mediante pagamento de uma retribuição específica, sob pena de arquivamento definitivo.\n#\n       Art. 34. Requerido o exame, deverão ser apresentados, no prazo de 60 (sessenta) dias, sempre que solicitado, sob pena de arquivamento do pedido:\n\n        I - objeções, buscas de anterioridade e resultados de exame para concessão de pedido correspondente em outros países, quando houver reivindicação de prioridade;\n\n        II - documentos necessários à regularização do processo e exame do pedido; e\n\n        III - tradução simples do documento hábil referido no § 2º do art. 16, caso esta tenha sido substituída pela declaração prevista no § 5º do mesmo artigo.\n#\n        Art. 35. Por ocasião do exame técnico, será elaborado o relatório de busca e parecer relativo a:\n\n   

In [9]:
len(chunks)

155

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vector_store = Chroma.from_documents(chunks, embeddings) 
retriever=vector_store.as_retriever() 

In [11]:
from langchain.chains import ConversationalRetrievalChain

crc = ConversationalRetrievalChain.from_llm(llm, retriever) 

In [12]:
question = 'o que é um certificado de adição ? qual artigo da LPI trata disso ?'
response = crc.invoke({'question':question,'chat_history':''})
print(response['answer'])

Um certificado de adição é um instrumento legal que protege aperfeiçoamentos ou desenvolvimentos introduzidos em uma invenção já patenteada, mesmo que esses aperfeiçoamentos não apresentem atividade inventiva, desde que estejam relacionados ao mesmo conceito inventivo da patente original. Ele é considerado acessório da patente e tem a mesma data de vigência.

O artigo da LPI (Lei de Propriedade Industrial) que trata do certificado de adição é o Art. 76.
